In [1]:
import math 
from ROOT import TMVA, TFile, TString
from array import array
import ROOT


Welcome to JupyROOT 6.18/00


In [2]:
%jsroot on

In [3]:
f1 = ROOT.TFile.Open('/eos/user/g/gtolkach/signal_event_with_cuts_plus_1.root')
f2 = ROOT.TFile.Open('/eos/user/g/gtolkach/background_event_with_cuts_plus_1.root')

signal = f1.Get('NOMINAL')
background = f2.Get('NOMINAL')

Warning in <TFile::Init>: file /eos/user/g/gtolkach/signal_event_with_cuts_plus_1.root probably not closed, trying to recover
Info in <TFile::Recover>: /eos/user/g/gtolkach/signal_event_with_cuts_plus_1.root, recovered key TTree:NOMINAL at address 5072325
Warning in <TFile::Init>: successfully recovered 1 keys
Warning in <TFile::Init>: no StreamerInfo found in /eos/user/g/gtolkach/signal_event_with_cuts_plus_1.root therefore preventing schema evolution when reading this file.


In [4]:
TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()
reader = TMVA.Reader( "!Color:!Silent" )


In [5]:

#branches = {}
#for branch in signal.GetListOfBranches():
#    branchName = branch.GetName()
#    branches[branchName] = array('f', [-999])
#    signal.SetBranchAddress(branchName, branches[branchName])
#    background.SetBranchAddress(branchName, branches[branchName])
#    if branchName != 'EventBranch':
#        reader.AddVariable(branchName, branches[branchName])

In [6]:
Pt = array('f',[0])
MetE = array('f',[0])
dPhi = array('f',[0])
PhiMiss = array('f',[0])
Mt = array('f',[0])
Eta = array('f',[0])

reader.AddVariable( "Pt", Pt)
background.SetBranchAddress("Pt", Pt )
signal.SetBranchAddress("Pt", Pt)
reader.AddVariable( "metE",MetE)
background.SetBranchAddress("metE", MetE )
signal.SetBranchAddress("metE", MetE)
reader.AddVariable( "dPhi",dPhi)
background.SetBranchAddress("dPhi", dPhi )
signal.SetBranchAddress("dPhi", dPhi)
reader.AddVariable( "PhiMiss",PhiMiss)
background.SetBranchAddress("PhiMiss", PhiMiss )
signal.SetBranchAddress("PhiMiss", PhiMiss)
reader.AddVariable( "Mt", Mt)
background.SetBranchAddress("Mt", Mt )
signal.SetBranchAddress("Mt", Mt)
reader.AddVariable( "Eta",Eta)
background.SetBranchAddress("Eta", Eta )
signal.SetBranchAddress("Eta", Eta)


4

In [7]:
reader.BookMVA("MLP",ROOT.TString("/eos/user/g/gtolkach/SWAN_projects/testpy/datasetV6newBDT/weights/TMVAClassification_MLP.weights.xml"))

<ROOT.TMVA::MethodMLP object ("MLP") at 0x6d1fc10>

                         : Booking "MLP" of type "MLP" from /eos/user/g/gtolkach/SWAN_projects/testpy/datasetV6newBDT/weights/TMVAClassification_MLP.weights.xml.
                         : Reading weight file: /eos/user/g/gtolkach/SWAN_projects/testpy/datasetV6newBDT/weights/TMVAClassification_MLP.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
<HEADER> MLP                      : Building Network. 
                         : Initializing weights
                         : Booked classifier "MLP" of type: "MLP"


In [8]:
hist_signal = ROOT.TH1D("output_s","Blue=Signal, Red=Background;EvaluateMVA;Probability", 20,0, 1.1)
hist_background = ROOT.TH1D("output_b","", 20, 0, 1.1)

In [9]:

for i in range(0,background.GetEntries()):
    background.GetEntry(i)
    Pt[0] = background.Pt
    MetE[0] = background.metE
    dPhi[0] = background.dPhi
    Mt[0] = background.Mt
    Eta[0] = background.Eta
    PhiMiss[0] = background.PhiMiss
    l = reader.EvaluateMVA("MLP")
    hist_background.Fill(l)
   


In [10]:

for i in range(0, signal.GetEntries()):
    signal.GetEntry(i)
    Pt[0] = signal.Pt
    MetE[0] = signal.metE
    dPhi[0] = signal.dPhi
    Mt[0] = signal.Mt
    Eta[0] = signal.Eta
    PhiMiss[0] = signal.PhiMiss
    k = reader.EvaluateMVA("MLP")
    hist_signal.Fill(k)    


In [11]:
hist_signal.Scale(1./hist_signal.Integral())
hist_background.Scale(1./hist_background.Integral())

In [12]:
c1 = ROOT.TCanvas() 
c1.cd()

hist_background.SetLineColor(ROOT.kRed)
hist_background.Draw("hist ")
hist_signal.Draw("hist same")
c1.Draw()